# Foundations of Data Science Project
## Phase 3: Sentiment Analysis
### By Arad Mahdinejad Kashani - 400102028
#### February 2024

*Apply the best classifiers of the second part on the dataset of the first part.*

The best models from the last phase were the `SpaCy` and `DISTIL_BERT` models. They will be loaded from the previous phase.

**Loading the data:**

In [2]:
import pandas as pd

In [3]:
tweets = pd.read_csv('/kaggle/input/zenododownloader/tweets.csv', on_bad_lines='skip')
tweets

,id,text,user_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_user_id,lang,source,created_at
0,865326129644797957,RT @cruzfloresiv: To all the weak hands who so...,859451814940336128,0,0,865323140531896320,3005609114,en,"<a href=""http://twitter.com/download/iphone"" r...",Thu May 18 22:00:00 +0000 2017
1,865326133008642049,"Investors Eye Fed, But Bond ETFs Still Add Ass...",44060322,0,0,0,0,en,"<a href=""https://about.twitter.com/products/tw...",Thu May 18 22:00:01 +0000 2017
2,865326134262681600,Sell $NLNK (Ne❑wLink Genetics Corporation) tha...,713570637306986496,0,0,0,0,en,"<a href=""https://u.9999yea.rs/bots/"" rel=""nofo...",Thu May 18 22:00:02 +0000 2017
3,865326134917050368,Increase: $VCO $NTES $BPT $ENIC $QIWI $JP $STX...,823239593151655936,0,0,0,0,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Thu May 18 22:00:02 +0000 2017
4,865326135952855040,"Former #FDA commissioner Califf joins Verily, ...",44367489,0,0,0,0,en,"<a href=""https://about.twitter.com/products/tw...",Thu May 18 22:00:02 +0000 2017
...,...,...,...,...,...,...,...,...,...,...
9091534,907725480618610689,Contrasting Pure Cycle Corporation $PCYO &amp;...,2920258887,0,0,0,0,pt,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Tue Sep 12 21:59:53 +0000 2017
9091535,907725484909309957,Contrasting Harvard Bioscience $HBIO &amp; Ful...,2920258887,0,0,0,0,da,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Tue Sep 12 21:59:54 +0000 2017
9091536,907725487996272640,"NRG Energy, Inc. $NRG Receives $21.13 Average ...",2920258887,0,0,0,0,en,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Tue Sep 12 21:59:55 +0000 2017
9091537,907725490760376320,Reviewing Arch Therapeutics $ARTH and Globus M...,2920258887,0,0,0,0,en,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Tue Sep 12 21:59:56 +0000 2017


In [4]:
tweets['ticker'] = tweets['text'].str.findall(r'\$(\w+)')
tweets['ticker'] = tweets['ticker'].apply(' '.join)
tweets['ticker']

0                                                        ltc
1                                                    IEF TLT
2                                                       NLNK
3          VCO NTES BPT ENIC QIWI JP STX DL LRCX SIMO LEA...
4                                                       GOOG
                                 ...                        
9091534                                             PCYO SBS
9091535                                            HBIO FLGT
9091536                                               NRG 21
9091537                                            ARTH GMED
9091538                                                   JD
Name: ticker, Length: 9091539, dtype: object

In [5]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

path = '/kaggle/input/fds-project-phase-2/DISTIL_BERT_results/'
distilbert = DistilBertForSequenceClassification.from_pretrained(path)

# Load the tokenizer
distilbert_tokenizer = DistilBertTokenizer.from_pretrained(path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
distilbert = distilbert.to(device)

In [6]:
device

device(type='cuda')

In [7]:
sample_size=100_000
df_sampled = tweets.sample(n=sample_size)
df_sampled

,id,text,user_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_user_id,lang,source,created_at,ticker
7881174,902573926684827648,$JPM J.P. Morgan Chase Has StalledOUR #1 PICK ...,586116334,0,0,0,0,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Tue Aug 29 16:49:27 +0000 2017,JPM AMTY
1620000,873249126078656514,At Home Group Inc $HOME Forecasted to Earn Q4 ...,828747327208841216,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Fri Jun 09 18:43:10 +0000 2017,HOME 0
1392075,872384105354321921,"Putnam FL Investment Management Co. Has $363,0...",1979190776,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Wed Jun 07 09:25:53 +0000 2017,363 ARCC
942507,870265816650002432,$OCRX Company has begun dosing of the first pa...,3331555937,0,0,0,0,en,"<a href=""https://stocknews.com/"" rel=""nofollow...",Thu Jun 01 13:08:34 +0000 2017,OCRX
2204812,876427424027881472,SG Americas Securities LLC Cuts Position in Ve...,849417718642159616,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Sun Jun 18 13:12:35 +0000 2017,VGR
...,...,...,...,...,...,...,...,...,...,...,...
2320530,876987382565789700,RT @bovosh: Good news for #Potcoin to sign a f...,2249186018,0,0,876971639539412992,2322596358,en,"<a href=""http://twitter.com/download/iphone"" r...",Tue Jun 20 02:17:40 +0000 2017,BTC ETC LTC
8568667,905702319978938368,Zacks: Brokerages Anticipate Armstrong Floorin...,68559732,0,0,0,0,en,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Thu Sep 07 08:00:34 +0000 2017,AFI 296
3224767,881053071895404544,RT @MOMOSTOCKTRADES: $VDRM $ELED $AMD $NVDA $D...,837341853922037760,0,0,879763780451926016,234956680,pl,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Sat Jul 01 07:33:16 +0000 2017,VDRM ELED AMD NVDA DCTH INTC QQQ ZNGA BBRY YHO...
6744205,897459678086561792,Analysts Expect Bruker Corporation $BRKR Will ...,68559732,0,0,0,0,en,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Tue Aug 15 14:07:15 +0000 2017,BRKR 414


In [8]:
from dateutil import parser
import pytz

def convert_to_datetime(dt_str):
    # Remove timezone from string
    dt_str = dt_str.rsplit(" ", 1)[0]
    # Convert string to datetime
    dt = parser.parse(dt_str)
    # Localize to PDT
    dt = dt.replace(tzinfo=pytz.timezone('US/Pacific'))
    return dt

# Apply the function to the 'datetime' column
df_sampled['datetime'] = df_sampled['created_at'].apply(convert_to_datetime)

df_sampled.head()

,id,text,user_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_user_id,lang,source,created_at,ticker,datetime
7881174,902573926684827648,$JPM J.P. Morgan Chase Has StalledOUR #1 PICK ...,586116334,0,0,0,0,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Tue Aug 29 16:49:27 +0000 2017,JPM AMTY,2024-08-29 17:42:27-07:00
1620000,873249126078656514,At Home Group Inc $HOME Forecasted to Earn Q4 ...,828747327208841216,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Fri Jun 09 18:43:10 +0000 2017,HOME 0,2024-06-09 19:36:10-07:00
1392075,872384105354321921,"Putnam FL Investment Management Co. Has $363,0...",1979190776,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Wed Jun 07 09:25:53 +0000 2017,363 ARCC,2024-06-07 10:18:53-07:00
942507,870265816650002432,$OCRX Company has begun dosing of the first pa...,3331555937,0,0,0,0,en,"<a href=""https://stocknews.com/"" rel=""nofollow...",Thu Jun 01 13:08:34 +0000 2017,OCRX,2024-06-01 14:01:34-07:00
2204812,876427424027881472,SG Americas Securities LLC Cuts Position in Ve...,849417718642159616,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Sun Jun 18 13:12:35 +0000 2017,VGR,2024-06-18 14:05:35-07:00


In [9]:
%pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 2.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541529 sha256=cde95e9157b6a6a29cbff17f8f21b1c078b6427ef0899d55aa2791f27e0ccfbd
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja
Note: you may need to restart the kernel to use updated packages.


In [10]:
import re
import string
from nltk.corpus import stopwords
import wordninja
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
# We want to remove URLs - @ and # signs - stopwords such as: 'is', 'at', 'which, digits and non-ASCII letters

def preprocess_tweet(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'\@\w+','', tweet)
    
    hashtags = re.findall(r"\#(\w+)", tweet)
    
    split_hashtags = [' '.join(wordninja.split(hashtag)) for hashtag in hashtags]
    
    for original, split in zip(hashtags, split_hashtags):
        tweet = tweet.replace('#' + original, split)

    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = re.sub('[^a-zA-Z]', ' ', tweet)
    
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words and w.isascii()]


    return (" ".join(filtered_words)).lower()

df_sampled['tweet_filtered'] = df_sampled['text'].apply(preprocess_tweet)
df_sampled

,id,text,user_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_user_id,lang,source,created_at,ticker,datetime,tweet_filtered
7881174,902573926684827648,$JPM J.P. Morgan Chase Has StalledOUR #1 PICK ...,586116334,0,0,0,0,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Tue Aug 29 16:49:27 +0000 2017,JPM AMTY,2024-08-29 17:42:27-07:00,jpm jp morgan chase has stalledour pick amty l...
1620000,873249126078656514,At Home Group Inc $HOME Forecasted to Earn Q4 ...,828747327208841216,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Fri Jun 09 18:43:10 +0000 2017,HOME 0,2024-06-09 19:36:10-07:00,at home group inc home forecasted earn q earni...
1392075,872384105354321921,"Putnam FL Investment Management Co. Has $363,0...",1979190776,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Wed Jun 07 09:25:53 +0000 2017,363 ARCC,2024-06-07 10:18:53-07:00,putnam fl investment management co has positio...
942507,870265816650002432,$OCRX Company has begun dosing of the first pa...,3331555937,0,0,0,0,en,"<a href=""https://stocknews.com/"" rel=""nofollow...",Thu Jun 01 13:08:34 +0000 2017,OCRX,2024-06-01 14:01:34-07:00,ocrx company begun dosing first patients part ...
2204812,876427424027881472,SG Americas Securities LLC Cuts Position in Ve...,849417718642159616,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Sun Jun 18 13:12:35 +0000 2017,VGR,2024-06-18 14:05:35-07:00,sg americas securities llc cuts position vecto...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320530,876987382565789700,RT @bovosh: Good news for #Potcoin to sign a f...,2249186018,0,0,876971639539412992,2322596358,en,"<a href=""http://twitter.com/download/iphone"" r...",Tue Jun 20 02:17:40 +0000 2017,BTC ETC LTC,2024-06-20 03:10:40-07:00,rt good news pot coin sign dispensaries in nev...
8568667,905702319978938368,Zacks: Brokerages Anticipate Armstrong Floorin...,68559732,0,0,0,0,en,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Thu Sep 07 08:00:34 +0000 2017,AFI 296,2024-09-07 08:53:34-07:00,zacks brokerages anticipate armstrong flooring...
3224767,881053071895404544,RT @MOMOSTOCKTRADES: $VDRM $ELED $AMD $NVDA $D...,837341853922037760,0,0,879763780451926016,234956680,pl,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Sat Jul 01 07:33:16 +0000 2017,VDRM ELED AMD NVDA DCTH INTC QQQ ZNGA BBRY YHO...,2024-07-01 08:26:16-07:00,rt vdrm eled amd nvda dcth intc qqq znga bbry ...
6744205,897459678086561792,Analysts Expect Bruker Corporation $BRKR Will ...,68559732,0,0,0,0,en,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Tue Aug 15 14:07:15 +0000 2017,BRKR 414,2024-08-15 15:00:15-07:00,analysts expect bruker corporation brkr will a...


In [11]:
global_counter = 0
total = len(df_sampled)

def predict_distilbert(text):
    every = 5000
    global global_counter
    # Ensure the model is in evaluation mode
    distilbert.eval()
    with torch.no_grad():
        inputs = distilbert_tokenizer(text, return_tensors='pt').to(device)
        outputs = distilbert(**inputs)
        # Get the predicted class index
        _, preds = torch.max(outputs.logits, dim=1)
        if global_counter % every == 0:
            print(f'{global_counter}/{total}')
        global_counter += 1
        return preds.item()

df_sampled['distilbert_prediction'] = df_sampled['tweet_filtered'].apply(lambda x: predict_distilbert(x))
df_sampled.head()

0/100000
5000/100000
10000/100000
15000/100000
20000/100000
25000/100000
30000/100000
35000/100000
40000/100000
45000/100000
50000/100000
55000/100000
60000/100000
65000/100000
70000/100000
75000/100000
80000/100000
85000/100000
90000/100000
95000/100000


,id,text,user_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_user_id,lang,source,created_at,ticker,datetime,tweet_filtered,distilbert_prediction
7881174,902573926684827648,$JPM J.P. Morgan Chase Has StalledOUR #1 PICK ...,586116334,0,0,0,0,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Tue Aug 29 16:49:27 +0000 2017,JPM AMTY,2024-08-29 17:42:27-07:00,jpm jp morgan chase has stalledour pick amty l...,0
1620000,873249126078656514,At Home Group Inc $HOME Forecasted to Earn Q4 ...,828747327208841216,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Fri Jun 09 18:43:10 +0000 2017,HOME 0,2024-06-09 19:36:10-07:00,at home group inc home forecasted earn q earni...,0
1392075,872384105354321921,"Putnam FL Investment Management Co. Has $363,0...",1979190776,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Wed Jun 07 09:25:53 +0000 2017,363 ARCC,2024-06-07 10:18:53-07:00,putnam fl investment management co has positio...,0
942507,870265816650002432,$OCRX Company has begun dosing of the first pa...,3331555937,0,0,0,0,en,"<a href=""https://stocknews.com/"" rel=""nofollow...",Thu Jun 01 13:08:34 +0000 2017,OCRX,2024-06-01 14:01:34-07:00,ocrx company begun dosing first patients part ...,0
2204812,876427424027881472,SG Americas Securities LLC Cuts Position in Ve...,849417718642159616,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Sun Jun 18 13:12:35 +0000 2017,VGR,2024-06-18 14:05:35-07:00,sg americas securities llc cuts position vecto...,0


What I did on the cell above is a python crime but it really does make sense when I am using `apply`.

In [12]:
global_counter = 0
total = len(df_sampled)

def modify_distilbert(n):
    every = 5000
    global global_counter

    if global_counter % every == 0:
        print(f'{global_counter}/{total}')
    global_counter += 1
    
    if n == 1:
        return 'POSITIVE'
    return 'NEGATIVE'

df_sampled['distilbert_prediction'] = df_sampled['distilbert_prediction'].apply(modify_distilbert)
df_sampled.head()

0/100000
5000/100000
10000/100000
15000/100000
20000/100000
25000/100000
30000/100000
35000/100000
40000/100000
45000/100000
50000/100000
55000/100000
60000/100000
65000/100000
70000/100000
75000/100000
80000/100000
85000/100000
90000/100000
95000/100000


,id,text,user_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_user_id,lang,source,created_at,ticker,datetime,tweet_filtered,distilbert_prediction
7881174,902573926684827648,$JPM J.P. Morgan Chase Has StalledOUR #1 PICK ...,586116334,0,0,0,0,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Tue Aug 29 16:49:27 +0000 2017,JPM AMTY,2024-08-29 17:42:27-07:00,jpm jp morgan chase has stalledour pick amty l...,NEGATIVE
1620000,873249126078656514,At Home Group Inc $HOME Forecasted to Earn Q4 ...,828747327208841216,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Fri Jun 09 18:43:10 +0000 2017,HOME 0,2024-06-09 19:36:10-07:00,at home group inc home forecasted earn q earni...,NEGATIVE
1392075,872384105354321921,"Putnam FL Investment Management Co. Has $363,0...",1979190776,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Wed Jun 07 09:25:53 +0000 2017,363 ARCC,2024-06-07 10:18:53-07:00,putnam fl investment management co has positio...,NEGATIVE
942507,870265816650002432,$OCRX Company has begun dosing of the first pa...,3331555937,0,0,0,0,en,"<a href=""https://stocknews.com/"" rel=""nofollow...",Thu Jun 01 13:08:34 +0000 2017,OCRX,2024-06-01 14:01:34-07:00,ocrx company begun dosing first patients part ...,NEGATIVE
2204812,876427424027881472,SG Americas Securities LLC Cuts Position in Ve...,849417718642159616,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Sun Jun 18 13:12:35 +0000 2017,VGR,2024-06-18 14:05:35-07:00,sg americas securities llc cuts position vecto...,NEGATIVE


In [13]:
import spacy
path = '/kaggle/input/fds-project-phase-2/SpaCy/'
nlp = spacy.load(path)

In [14]:
global_counter = 0
total = len(df_sampled)

def predict_spacy(text):
    every = 5000
    global global_counter

    if global_counter % every == 0:
        print(f'{global_counter}/{total}')
    global_counter += 1
    
    doc = nlp(text)
    
    return max(doc.cats, key=doc.cats.get)

df_sampled['spaCy_prediction'] = df_sampled['tweet_filtered'].apply(lambda x: predict_spacy(x))
df_sampled.head()

0/100000
5000/100000
10000/100000
15000/100000
20000/100000
25000/100000
30000/100000
35000/100000
40000/100000
45000/100000
50000/100000
55000/100000
60000/100000
65000/100000
70000/100000
75000/100000
80000/100000
85000/100000
90000/100000
95000/100000


,id,text,user_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_user_id,lang,source,created_at,ticker,datetime,tweet_filtered,distilbert_prediction,spaCy_prediction
7881174,902573926684827648,$JPM J.P. Morgan Chase Has StalledOUR #1 PICK ...,586116334,0,0,0,0,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Tue Aug 29 16:49:27 +0000 2017,JPM AMTY,2024-08-29 17:42:27-07:00,jpm jp morgan chase has stalledour pick amty l...,NEGATIVE,NEGATIVE
1620000,873249126078656514,At Home Group Inc $HOME Forecasted to Earn Q4 ...,828747327208841216,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Fri Jun 09 18:43:10 +0000 2017,HOME 0,2024-06-09 19:36:10-07:00,at home group inc home forecasted earn q earni...,NEGATIVE,POSITIVE
1392075,872384105354321921,"Putnam FL Investment Management Co. Has $363,0...",1979190776,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Wed Jun 07 09:25:53 +0000 2017,363 ARCC,2024-06-07 10:18:53-07:00,putnam fl investment management co has positio...,NEGATIVE,NEGATIVE
942507,870265816650002432,$OCRX Company has begun dosing of the first pa...,3331555937,0,0,0,0,en,"<a href=""https://stocknews.com/"" rel=""nofollow...",Thu Jun 01 13:08:34 +0000 2017,OCRX,2024-06-01 14:01:34-07:00,ocrx company begun dosing first patients part ...,NEGATIVE,POSITIVE
2204812,876427424027881472,SG Americas Securities LLC Cuts Position in Ve...,849417718642159616,0,0,0,0,en,"<a href=""https://zapier.com/"" rel=""nofollow"">Z...",Sun Jun 18 13:12:35 +0000 2017,VGR,2024-06-18 14:05:35-07:00,sg americas securities llc cuts position vecto...,NEGATIVE,POSITIVE


*Using different correlation measures (at least 2 and you can use libraries), find the correlation between the sentiments of each CashTag and its value (from the financial data in the companies dataset) in a time interval.*

In [30]:
companies = pd.read_csv('/kaggle/input/zenododownloader/companies.csv', on_bad_lines='skip')
companies.columns

Index(['ticker', 'name', 'exchange', 'capitalization'], dtype='object')

In [34]:
from scipy.stats import pearsonr, spearmanr

#df_sampled['day'] = df_sampled['day'].dt.to_timestamp()
# Calculate the average sentiment per day for each CashTag
avg_sentiment = df_sampled.groupby(['ticker', 'day'])[['distilbert_prediction_num', 'spaCy_prediction_num']].mean().reset_index()

# Drop rows with NaN values
companies_clean = companies.dropna()

# Merge the average sentiment data with the companies data
merged_data = pd.merge(avg_sentiment, companies_clean, on=['ticker'])

# Select a specific time interval
middle_month = merged_data['day'].min() + (merged_data['day'].max() - merged_data['day'].min()) / 2
middle_month = middle_month.to_period('M').to_timestamp()
middle_data = merged_data[(merged_data['day'] >= middle_month) & (merged_data['day'] < middle_month + pd.DateOffset(months=1))]

# Calculate the correlation for the selected interval
pearson_corr_distilbert, _ = pearsonr(middle_data['capitalization'], middle_data['distilbert_prediction_num'])
spearman_corr_distilbert, _ = spearmanr(middle_data['capitalization'], middle_data['distilbert_prediction_num'])

pearson_corr_spaCy, _ = pearsonr(middle_data['capitalization'], middle_data['spaCy_prediction_num'])
spearman_corr_spaCy, _ = spearmanr(middle_data['capitalization'], middle_data['spaCy_prediction_num'])

print(f'Pearson correlation between capitalization and DistilBERT sentiment for the middle month: {pearson_corr_distilbert}')
print(f'Spearman correlation between capitalization and DistilBERT sentiment for the middle month: {spearman_corr_distilbert}')

print(f'Pearson correlation between capitalization and spaCy sentiment for the middle month: {pearson_corr_spaCy}')
print(f'Spearman correlation between capitalization and spaCy sentiment for the middle month: {spearman_corr_spaCy}')

Pearson correlation between capitalization and DistilBERT sentiment for the middle month: -0.005229084784529485
Spearman correlation between capitalization and DistilBERT sentiment for the middle month: -0.012521002314906519
Pearson correlation between capitalization and spaCy sentiment for the middle month: 0.004165189124265656
Spearman correlation between capitalization and spaCy sentiment for the middle month: -0.029781320887004303


*Using different correlation measures, find the correlation between change of sentiments of CashTag and the number of tweets related to that CashTag in the time series (By time series of tweets, we mean the number of tweets per given time that was calculated in the first part).*

In [35]:
df_sampled['day'] = df_sampled['datetime'].dt.to_period('D')

# Convert the sentiment predictions to numerical values
df_sampled['distilbert_prediction_num'] = df_sampled['distilbert_prediction'].map({'POSITIVE': 1, 'NEGATIVE': -1})
df_sampled['spaCy_prediction_num'] = df_sampled['spaCy_prediction'].map({'POSITIVE': 1, 'NEGATIVE': -1})

# Calculate the average sentiment per day for each CashTag
avg_sentiment = df_sampled.groupby(['ticker', 'day'])[['distilbert_prediction_num', 'spaCy_prediction_num']].mean().reset_index()
tweet_counts = df_sampled.groupby(['ticker', 'day'])['id'].count().reset_index().rename(columns={'id': 'tweet_count'})

# Merge the average sentiment data with the tweet_counts data
merged_data = pd.merge(tweet_counts, avg_sentiment, on=['ticker', 'day'])

# Calculate the correlation
correlation_distilbert = merged_data['tweet_count'].corr(merged_data['distilbert_prediction_num'])
correlation_spaCy = merged_data['tweet_count'].corr(merged_data['spaCy_prediction_num'])

print(f'Correlation between tweet count and DistilBERT sentiment: {correlation_distilbert}')
print(f'Correlation between tweet count and spaCy sentiment: {correlation_spaCy}')

/tmp/ipykernel_34/740084830.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_sampled['day'] = df_sampled['datetime'].dt.to_period('D')


Correlation between tweet count and DistilBERT sentiment: 0.003474215296178894
Correlation between tweet count and spaCy sentiment: -0.004355228140295381


*Find examples where the classifiers do not agree on the sentiment of the tweet and analyze the results and discuss where does each classifier make mistakes.*

In [36]:
# Find examples where the classifiers do not agree
disagreements = df_sampled[df_sampled['distilbert_prediction'] != df_sampled['spaCy_prediction']]


n = 5
# Print some examples
for idx, row in disagreements[['text', 'distilbert_prediction', 'spaCy_prediction']].head(n).iterrows():
    print(f"Text: {row['text']}")
    print(f"DistilBERT Prediction: {row['distilbert_prediction']}")
    print(f"spaCy Prediction: {row['spaCy_prediction']}")
    print("\n")

Text: At Home Group Inc $HOME Forecasted to Earn Q4 2018 Earnings of $0.30 Per Share https://t.co/VDycVabjjq
DistilBERT Prediction: NEGATIVE
spaCy Prediction: POSITIVE


Text: $OCRX Company has begun dosing of the first patients in Part Two of a Phase1/Phase 2a clinical trial of oral OCR-0… https://t.co/csMCRZ2oBq
DistilBERT Prediction: NEGATIVE
spaCy Prediction: POSITIVE


Text: SG Americas Securities LLC Cuts Position in Vector Group Ltd. $VGR https://t.co/7Rp2CfVfOo
DistilBERT Prediction: NEGATIVE
spaCy Prediction: POSITIVE


Text: Dimensional Fund Advisors LP Has $15.57 Million Position in Capital City Bank Group $CCBG https://t.co/x5hl85YeKQ
DistilBERT Prediction: POSITIVE
spaCy Prediction: NEGATIVE


Text: Startin the week off with a bang. $RGNX https://t.co/AX5RTRcGNh TV_TradingIdeas
DistilBERT Prediction: POSITIVE
spaCy Prediction: NEGATIVE




*Discuss transfer learning and fine-tuning and how they can be used to improve the overall effect of the project. Use your limited results from the first part.*

**Transfer Learning** and **Fine-Tuning** are powerful techniques in machine learning that can significantly improve the performance of a model, especially when dealing with tasks where the available data is limited.

**Transfer Learning** is a technique where a pre-trained model, which has been trained on a large-scale dataset, is used as a starting point for a related task. The idea is to leverage the knowledge that the model has already learned from the large dataset to perform the new task. This is particularly useful when the new task has limited data, as it allows the model to generalize better.

For example, in the context of this project, a pre-trained language model like BERT or DistilBERT, which has been trained on a large corpus of text, can be used as a starting point for sentiment analysis. The pre-trained model already understands the semantics of the language to a great extent, which can be leveraged to understand the sentiment of the tweets.

**Fine-Tuning** is a process that adjusts the pre-trained model to the new task. After initializing our model with the pre-trained weights, we continue training the model on our specific task (like sentiment analysis in this project). During this process, the model learns to adjust its weights and biases to specialize on the new task.

In the context of this project, fine-tuning can be used to adapt the pre-trained language model to the specific language and style used in the tweets. This can lead to a more accurate sentiment prediction.

In summary, transfer learning and fine-tuning can greatly improve the performance of the model in this project by leveraging the knowledge from large-scale pre-training and adapting it to the specific task of tweet sentiment analysis. This can lead to more accurate sentiment predictions, which in turn can improve the correlation analysis between tweet sentiments and company values.

We can check how the sentiment and amount of tweets change drastically in a given time to loosely predict how the stock and capitalization of companies will in turn, change. This is especially useful in trading and forecasting the market. For example in the first part the time series plot of tweets increase when the market reports arrive, and we can see peaks. The sentiments may be mixed, however. Sentiments do not perform that well when used for predicting the market (as our analysis showed). This is especially important as piggybacking may occur and bots may be involved to change the public views on some stock so that people will buy them.